# Explorer l'évolution d'un paysage avec FastScape

Fastscape (https://fastscape.org) est une série de programmes permettant de modéliser l'évolution topographique d'une région, en prenant en comptes des paramètres tels que vitesse de surrection, dureté des roches, etc.

Il est fourni sous la forme d'un certain nombre de modules en Python. 

Ce fichier présente un exemple très simple pour explorer quelques paramètres.

Il faut exécuter successivement les différentes commandes : placez vous dans chacune des boîtes de code et appuyez sur ctrl+entrée pour exécuter le code (dans l'ordre, de haut en bas). Le résultat apparaît en dessous.

Vous pouvez modifier le code pour expérimenter. En particulier un peu plus bas, nous verrons la signification de quelques paramètres clef.

Le premier morceau de code, ci-dessous, permet de charger les bibliothèques nécessaires. Exécutez le une seule fois en début de session.

In [1]:
import numpy as np
import xsimlab as xs
import xarray as xr

from fastscape.models import basic_model
from fastscape.models import sediment_model

from ipyfastscape import TopoViz3d

Le code suivant crée un modèle, qui inclue de l'érosion et du transport de sédiments, dans un paysage qui a déjà un escarpement présent et dont une partie est surrélevée ("uplift"). Executez-le une seule fois.

In [2]:
from fastscape.processes import Escarpment, TwoBlocksUplift

model = (sediment_model
         .update_processes({'init_topography': Escarpment,
                                   'uplift': TwoBlocksUplift
}))

# Paramètres

C'est cette partie du code qui devient intéressante ! Elle définit les paramètres de la simulation. Vous pouvez les explorer et les modifier; après chaque modification, executez cette partie du code et les suivantes.

Utilisez la notation scientifique pour toutes les valeurs : il faut mettre `1e1` et pas 10.

## clocks
Modifier ce paramètre pour changer la durée de la simulation, sous la forme (début, fin, longeur de chaque pas). Par exemple `(0, 4e5 + 2e3, 2e3)` signifie de 0 à 402 000 ans, avec des pas de 200 ans. Attention, plus vous rajoutez de pas, plus le calcul sera long...

## grid__shape
Nombre de cellules dans la grille, en x et en y. Là aussi plus il y a de cellules, plus le calcul sera long...

## grid__length
Taille de la grille, en mètres (x et y). `[1e4, 2e4]` signifie donc une grille de 10 x 20 km, et si vous avez défini 100 x 200 cellules ça implique que chaque cellule fait 100 x 100 m.

## boundary__status
A priori ne pas modifier.

Conditions aux limites (looped, core, fixed_value). Dans l'ordre gauche, droite, bas, haut. 
(au moins une borduire doit être fixe, `core` signifie une bordure libre, `looped` s'utilise deux par deux pour avoir des bordures identitiques par paire)

## init_topography
La topographie initiale contient un escarpement plus ou moins en pente (il commence à `x_left` et finit à `x_right`, si les deux sont égaux il est vertical), séparant un compartiment bas à gauche (`elevation_left`) et un compartiment haut à droite (`elevation_right`). En mètres.

## uplift
Les vitesses de surrection. Elles sont différentes de part et d'autre d'une limite située à `x_position`. Le taux de surrection (en m/an) est différent à gauche et à droite de cette limite (`rate_left`, `rate_right`).

Une vitesse de surrection typique est de 0.1 à 1 mm/an (`1e-4` à `1e-3`)

## Lois d'érosion
Cette série de paramères contrôle la façon dont les roches sont érodées et transportées. Elle comporte trois composants:

### 1) Direction de transport
Sans doute peu important.
`flow__slope_exp` peut êre réglé à une valeur entre 0 et 1. Avec une valeur de 0 le flux ignore la pente (les sédiments se déplacement vers tous les voisins), avec 1 il est multi-directionel (la matière est transportée vers plusieurs cellules en aval), avec des valeurs plus élevées il est fortement focalisé dans une seule direction (la plus grande pente).

### 2) Erosivité
La vitesse de creusement par les cours d'eau est modelisée différement pour les roches en place ("bedrock") et les sédiments ("soil"). Le coefficient k indique la facilité d'érosion (donc des matériaux à k élevé sont plus tendre). L'érosion dépend aussi de la pente et de la surface drainée, chacune affectée d'un exposant (donc 1 = loi linéaire): `area_exp` et `slope_exp`. 

Les coefficients g est relié à la proportion de matériaux qui se déposent (par opposition à ceux transportés plus loin par la rivière). Avec g=0 tous les matériaux érodés sont évacués, g=1 permet de les redéposer en amont.

### 3) Mouvements de pente
En dehors des rivières, de la matière peut être transportée sur les pentes, de façon diffuse (modélise les glissements de terrain, les chutes de blocs des falaises, etc.). Les deux coefficients de diffusivité indiquent l'importance de ce processus.

In [46]:
in_ds = xs.create_setup(
    model=model,
    clocks={
        'time': np.arange(0, 4e5 + 2e3, 2e3),
    },
    master_clock='time',
    input_vars={
        'grid__shape': [101, 201],
        'grid__length': [1e4, 2e4],
        'boundary__status': ['looped', 'looped', 'fixed_value', 'core'],
        'init_topography': {
            'x_left': 1e4,
            'x_right': 1e4,
            'elevation_left': 1e2,
            'elevation_right': 2e2
        },
        'uplift': {
            'x_position': 1e4,
            'rate_left': 5e-4,
            'rate_right': 1e-3
        },
        'flow__slope_exp': 1.,
        'spl': {
            'k_coef_bedrock': 1e-4,
            'k_coef_soil': 1.5e-4,
            'g_coef_bedrock': 1.,
            'g_coef_soil': 1.,
            'area_exp': 0.4,
            'slope_exp': 1.
        },
        'diffusion': {
            'diffusivity_bedrock': 1e-2,
            'diffusivity_soil': 1.5e-2
        }
    },
    output_vars={
        'topography__elevation': 'time',
        'erosion__rate': 'time'
    }
)

# Exécution !

Executez la cellule suivante pour calculer le modèle. Selon vos paramètres, il peut prendre de quelques secondes à plusieurs minutes.

In [47]:
with xs.monitoring.ProgressBar():
    out_ds = in_ds.xsimlab.run(model=model)

             0% | initialize 

# Visualisation
Exécutez la cellule ci-dessous pour faire apparaître un gadget interactif qui vous permettra d'explorer votre modèle et son comportement au cours du temps...

In [48]:
app = TopoViz3d(out_ds, canvas_height=600, time_dim="time")

app.show()

Output(layout=Layout(height='640px'))